In [139]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [86]:
x_data = np.load('../2-Processing/x_data.npy')
y_data = np.load('../2-Processing/y_data.npy')
x_test = np.load('../2-Processing/x_test.npy')

In [87]:
x_data.shape, y_data.shape, x_test.shape

((45532, 21), (45532, 1), (11383, 21))

In [98]:
y_data = np.reshape(y_data, (len(y_data), ))

In [101]:
x1, y1 = x_data[y_data==1], y_data[y_data==1]
x0, y0 = x_data[y_data==0], y_data[y_data==1]

x1.shape, y1.shape, x0.shape, y0.shape

((20634, 21), (20634,), (24898, 21), (20634,))

In [110]:
import random
n = np.random.choice(24898, 20634, replace=False)

In [111]:
n

array([ 3841, 11294, 12561, ...,  4692,  1411, 15362])

In [88]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid=train_test_split(x_data, y_data, 
                                                    test_size=0.1, 
                                                    shuffle=True, 
                                                    random_state=1004)
x_train.shape, x_valid.shape

((40978, 21), (4554, 21))

In [120]:
y_train = np.reshape(y_train, (len(y_train), ))
y_valid = np.reshape(y_valid, (len(y_valid), ))

In [129]:
import lightgbm as lgbm
model = lgbm.LGBMClassifier(n_estimators=500)
model.fit(x_train, y_train, early_stopping_rounds=100, 
         eval_metric='auc',
         eval_set=(x_valid, y_valid), verbose=True)

[1]	valid_0's auc: 0.744898	valid_0's binary_logloss: 0.668936
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.747552	valid_0's binary_logloss: 0.653121
[3]	valid_0's auc: 0.748812	valid_0's binary_logloss: 0.640061
[4]	valid_0's auc: 0.749335	valid_0's binary_logloss: 0.629112
[5]	valid_0's auc: 0.749469	valid_0's binary_logloss: 0.619927
[6]	valid_0's auc: 0.750154	valid_0's binary_logloss: 0.612162
[7]	valid_0's auc: 0.750763	valid_0's binary_logloss: 0.60545
[8]	valid_0's auc: 0.750883	valid_0's binary_logloss: 0.599831
[9]	valid_0's auc: 0.750859	valid_0's binary_logloss: 0.595184
[10]	valid_0's auc: 0.751044	valid_0's binary_logloss: 0.591097
[11]	valid_0's auc: 0.751268	valid_0's binary_logloss: 0.587578
[12]	valid_0's auc: 0.75115	valid_0's binary_logloss: 0.584714
[13]	valid_0's auc: 0.751029	valid_0's binary_logloss: 0.582142
[14]	valid_0's auc: 0.751593	valid_0's binary_logloss: 0.579781
[15]	valid_0's auc: 0.751942	valid_0's binary_logloss

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [130]:
y_pred = model.predict(x_valid)

In [131]:
count=0
for y, y_h in zip(y_valid, y_pred):
    if y==y_h: count+=1
count, len(y_valid)

(3108, 4554)

In [132]:
count/len(y_valid)

0.6824769433465085

In [142]:
y_sub = model.predict(x_test)

In [144]:
y_sub[y_sub==0]=2
y_sub

array([2, 2, 2, ..., 1, 1, 1])

In [145]:
submission=pd.read_csv('../0-Data/org/sample_submission.csv', index_col=0)  

In [147]:
submission['voted']=y_sub

In [150]:
submission.to_csv('../0-Data/org/sample_submission.csv')